
<h2>Introduction</h2>

I would like to talk about some of the design principles in the <a href="https://github.com/WinVector/data_algebra"><code>data_algebra</code> package</a> (and also in its sibling <a href="https://github.com/WinVector/rquery"><code>rquery</code> package</a>).

<!--more--><p/>

<h2>Background</h2>

<h3>sklearn Pipeline</h3>

A major influence on the <code>data_algebra</code> design is <a href="https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline"><code>sklearn.pipeline.Pipeline</code></a>. <code>sklearn.pipeline.Pipeline</code> itself presumably become public with <a href="https://github.com/scikit-learn/scikit-learn/commit/b99c76550e3cbe8b57b1ea27b6eb88817a36cb53">Edouard Duchesnay's Jul 27, 2010 commit: "add pipeline"</a>.

<code>sklearn.pipeline.Pipeline</code> maintains a list of steps to be applied to data.  What is interesting is the steps are <em>not</em> functions. Steps are instead objects that implement both a <code>.transform()</code> and a <code>.fit()</code> method.

<code>.transform()</code> typically accepts a data-frame type structure and returns a modified version.  Typical operations include adding a new derived column, selected columns, selected rows, and so on.

From a transform-alone point of view the steps compose like functions.  For list <code>[s, t]</code> <code>transform(x)</code> is defined to as:

<pre><code>   transform([s, t], x) = 
      t.transform(s.transform(x))</pre></code>

The fit-perspective are where things get interesting.  <code>obj.fit(x)</code> changes the internal state of obj based on the value <code>x</code> and returns a reference to <code>obj</code>.  I.e. it learns from the data and stores this result as a side-effect in the object itself.  In <code>sklearn</code> it is common to assume a composite method called <code>.fit_transform()</code> defined as: 
<code><pre>   obj.fit_transform(x) := obj.fit(x).transform(x)</pre></code>
(the "<code>:=</code>" denoting "defined as").

Using <code>.fit_transform()</code> we can explain that in a <code>sklearn Pipeline</code> <code>.fit()</code> is naturally thought of as:
<code><pre>   fit([s, t], x]) = 
      t.fit(s.fit_transform(x))</pre></code>

My point is: <code>sklearn.pipeline.Pipeline</code> generalizes function composition (as we see with the <code>.transform()</code> methods) to something more powerful (the ability to both fit and to transform).  This becomes the natural way to store a sequence of parameterized or data-dependent data transform steps (such as centering, scaling, missing value imputation, and much more).

A rigid mindset where "function composition is the only form of composition" would not easily allow the above effects. Instead the composition idea is list concatenation or a free group (essentially the graduate school way of saying "list concatenation").

<h3>Fist class citizens</h3>

We are going to try to design our tools to be "<a href="https://en.wikipedia.org/wiki/First-class_citizen">first class citizens</a>" in the sense of Strachey:

<blockquote>
<strong>First and second class objects.</strong> In <code>ALGOL</code>, a real number may appear in an expression or be assigned to a variable, and either of them may appear as an actual parameter in a procedure call. A procedure, on the other hand, may only appear in another procedure call either as the operator (the most common case) or as one of the actual parameters. There are no other expressions involving procedures or whose results are procedures. Thus in a sense procedures in <code>ALGOL</code> are second class citizens—they always have to appear in person and can never be represented by a variable or expression (except in the case of a formal parameter)...
<p/>
<a href="https://en.wikipedia.org/wiki/First-class_citizen">Quoted in Wikipedia.
Christopher Strachey, "Fundamental Concepts in Programming Languages" in Higher-Order and Symbolic Computation 13:11 (2000); though published in 2000, these are notes from lectures Strachey delivered in August, 1967</a>
</blockquote>

What we will draw out: is if our data transform steps are "first class citizens" we should expect to be able to store them in variables, compose them, examine them, and many other steps.  A function that we can only use or even re-use is not giving us as much as we expect from other types.  Or alternately, if functions don't give us everything we want, we may not want to use them as our only type or abstraction of data processing steps.
  

<h3>Composability</h3>

Most people first encounter the mathematical concept of "composability" in terms of functions.  This can give the false impression that to work with composable design principles, one must shoe-horn the object of interest to be functions or some sort of augmented functions.

This Procrustean view loses a lot of design opportunities.

In mathematics composability is directly studied by the field called "<a href="https://en.wikipedia.org/wiki/Category_theory">Category Theory</a>." So it makes sense to see if category theory may have ideas, notations, tools, and results that may be of use.

<h2>Category Theory</h2>

A lot of the benefit of <a href="https://en.wikipedia.org/wiki/Category_theory">category theory</a> is lost if every time we try to apply category theory (or even just use some of the notational conventions) we attempt to explain <em>all</em> of category theory as a first step.  So I will try to resist that urge here.  I will introduce the bits I am going to use here.

Category theory routinely studies what are called "arrows."  When treated abstractly an arrow has two associated objects called the "domain" and "co-domain." The names are meant to be evocative of the "domain" (space of inputs) and "co-domains" (space of outputs) from the theory of functions.

Category theory differs from function theory in that category theory is careful to keep separate the following two concepts: what arrows are and how arrows are composed.

When using arrows to model a system we expect to be able to specify, with some degree of freedom:

<ul>
<li>Which arrow(s) are associated with a given domain and co-domain pair.</li>
<li>How arrows compose.  For arrows <code>a</code> and <code>b</code> with <code>co-domain(b) = domain(a)</code> then: <code>a . b</code> denotes the composition in the category, and is itself a new arrow in the same category.  Composition is not allowed (or defined) when <code>co-domain(b) != domain(a)</code>.</li>
<li>How arrows <a href="https://ncatlab.org/nlab/show/action">act</a> on items.</li> 
</ul>

An action is a mapping from arrows and items to items.  I.e. <code>action(arrow, item) = new_item</code>. For categories the items may or may not be related to the domain and co-domain. Not all categories have actions, but when they do have actions the action must be compatible with arrow composition.

Good general references on category theory, include:

<ul>
    <li>Steve Awodey, <em>Category Theory, 2nd Edition</em>, Oxford University Press; 2010.</li>
    <li>Emily Riehl, <em>Category Theory in Context</em>, Dover, 2016.</li>
    <li>Saunders Mac Lane, <em>Categories for the Working Mathematician, 2nd Edition</em>, Springer, 1978.</li>
</ul>


Functions have a very ready category theory interpretation as arrows.  We can treat a function <code>f</code> as an arrow with domain equal to the set of values the function is defined for (also called the domain of the function) with co-domain equal to the image of the domain (also called the range of a function), or any set containing the range of the function.  In this formulation we define the arrow composition of <code>f</code> and <code>g</code> as <code>f . g</code> is defined to be the function such that for all <code>x</code> in domain <code>x</code> we have:
<code><pre>   (f . g)(x) := f(g(x)) </pre></code>

We will call the application of a function to a value as an example of an "<a href="https://ncatlab.org/nlab/show/action">action</a>." A function <code>f()</code> "acts on its domain" and <code>f(x)</code> is the action of <code>f</code> on <code>x</code>.  For functions we can define the action "<code>apply</code>" as:
<code><pre>   apply(f, x) := f(x)</pre></code>

The extra generalization power we get from moving away from functions to arbitrary arrows (that might not correspond to functions) comes from the following:

<ul>
<li>Arrow composition does <em>not</em> have to be function composition.</li>
<li>Arrows can <a href="https://ncatlab.org/nlab/show/action">act</a> on items that are <em>not</em> elements of their domain.</li>
<li>Arrows have a notion of equality, but this notion <em>can differ</em> from having identical actions.  (Functions that have identical actions are usually considered to be identical by the <a href="https://en.wikipedia.org/wiki/Axiom_of_extensionality">axiom of extensionality</a>.)</li>
</ul>

To be a category a few conditions must be met, including: the composition must be associative and we must have some identity arrows. By "associative composition" we mean, it must be the case that for arrows <code>a</code>,
<code>b</code>, and <code>c</code> (with appropriate domains and co-domains):
<code><pre>   (a . b) . c = a . (b . c) </pre></code>

Our action must also associate with arrow composition.  That is we must have for values <code>x</code> we must have for covariant actions:
<code><pre>   apply(a . b, x) = apply(a, apply(b, x))</pre></code>

Or for contravariant actions:
<code><pre>   apply(a . b, x) = apply(b, apply(a, x))</pre></code>


The idea is: the arrow <code>a . b</code> must have an action equal to the actions of a and b composed as functions.

Arrow composition and actions can differ from function composition and function application, but they must be at least somewhat similar in that they remain <a href="https://en.wikipedia.org/wiki/Associative_property">associative</a>.

<h2>Back to <code>sklearn.pipeline.Pipeline</code></h2>

We now have enough notation to attempt a crude category theory description of <code>sklearn.pipeline.Pipeline</code>.

Define our <code>sklearn.pipeline.Pipeline</code> category <code>P</code> as follows:

<ul>
<li>We have only one object called <code>0</code>. All arrows will have domain and co-domain equal to <code>0</code>, i.e.: we are not doing any interesting pre-condition checking in this category. </li>
<li>The arrows of our category are lists of steps.  
Steps are again <code>Python</code> objects
that define <code>.transform()</code>, <code>.fit()</code>, and <code>.fit_transform()</code> methods.</li>
<li>Composition <code>a1 . a2</code> is defined as list concatenation <code>a2 + a1</code>.  "<code>+</code>" being <code>Python</code>'s list concatenate in this case, and the order set to match <code>sklearn.pipeline.Pipeline</code> list order convention.</li>
<li>We define an action called "<code>transform_action</code>" defined as:

<code><pre>   transform_action([step1, step2, ..., stepk], x) := 
      stepk.transform(... step2.transform(step1.transform(x)) )</pre></code>
</li>
</ul>

To see this is a category (and a category compatible action) we must check associativity of the composition (which in this case is list concatenation) and associativity of the action with respect to list concatenation.  

We can also try to model the <code>.fit_transform()</code> methods.  We will not try to model the side-effect that <code>.fit_transform()</code> changes state of the arrows (to have the fit information in each step).  But we can at least define an action (with side effects) as follows:

<ul>
<li>We define an action called "<code>fit_transform</code>" defined as:

<code><pre>   fit_transform_action([step1, step2, ..., stepk], x) := 
      stepk.fit_transform(... step2.fit_transform(step1.fit_transform(x)) )</pre></code>
</li>
</ul>

To confirm this is roughly an action, we would want check is if the following equality holds or not:
<code><pre>  fit_transform_action(a . b, x) =
      fit_transform_action(b, fit_transform_action(a, x))
   </pre></code>

The above should follow by brute pushing notation around (assuming we have defined <code>fit_transform_action</code> correctly, and sufficiently characterized <code>.fit_transform()</code>).

Notice we didn't define a "<code>fit_action</code>" action, as it isn't obvious that has a not obvious that has a nice associative realization. This an example of theory driving the design: <code>fit_transform()</code> may be more fundamental than, and thus preferred over, <code>fit()</code>, due to the easier to argue associativity of <code>fit_transform()</code>.

The category theory concepts didn't so-much design <code>sklearn.pipeline.Pipeline</code>, but give us a set of criteria to evaluate <code>sklearn.pipeline.Pipeline</code> design.  We trust the category theory point of view is useful as it emphasizes associativity (which is a great proprety to have), and is routinely found to be a set of choices that work in complicated systems.  The feeling being: the design points category theory seems to suggest, turn out to be the one you want down the round.

<h2>The <code>data_algebra</code></h2>

<h3>What is the <code>data_algebra</code>?</h3>

<ul>
<li>
    <a href="https://github.com/WinVector/data_algebra"><code>data_algebra</code></a> is a package for building up complex data manipulation queries
 <code>data_algebra</code> queries are first class citizens in the Strachey sense (can be: passed as an argument, returned from a function, modified, assigned to a variable, printed, inspected, and traversed as a data structure).
</li>
<li>
 The operators are essentially those of the Codd relational algebra (select rows/columns, join, unionall, extend, project, and window functions).
</li>
<li>
    Composition is left to right using method chaining.
</li>
<li>
    Queries can be realized in <codd>SQL</code> (targeting <code>PostgeSQL</code> and <code>Spark</code>) or in <code>Pandas</code> (hoping to extend to <code>modin</code>, <code>RAPIDS</code>, and others).
    </li>
    <li>The <code>data_algebra</code> has an <a href="https://www.r-project.org"><code>R</code></a> sibling package group
        (<a href="https://github.com/WinVector/rquery"><code>rquery</code></a>/<a href="https://github.com/WinVector/rqdatatable"><code>rqdatatable</code></a>) similar to <a href="https://CRAN.R-project.org/package=dplyr"><code>dplyr</code></a>.</li>
</ul>

An introduciton to the <code>data_algebra</code> can be found <a href="https://github.com/WinVector/data_algebra">here</a>.

We now have the terminology to concisely state a <code>data_algebra</code> design principle: use general concepts (such as category theory notation) to try and ensure <code>data_algebra</code> steps have a good description and are first class citizens (i.e. we can do a lot with them and to them).

<h3>The naive functional view</h3>

If we were to again take a mere functional view of the <a href="https://github.com/WinVector/data_algebra"><code>data_algebra</code></a> we would say the <code>data_algebra</code> is a set of functions that operate on data.  They translate data frames to new data frames using <a href="https://en.wikipedia.org/wiki/Relational_algebra">Codd]</a>-inspired operations. 

However, this is not correct.  <code>data_algebra</code> methods actually map data transforms to data tranforms.  We only apply them to data later.  However even this is a "too functional view" as the <code>data_algebra</code> compose a manner different than mere function composition.

<h3>The categorical view</h3>

The <code>data_algebra</code> can be mapped to a nice category.  The idea being something that can be easilly mapped to an orderly system, is it self likely an somewhat orderly system.

Good references on the application of category theory to concrete systems (including databases) include:

<ul>
    <li>David I. Spivak, <em>Category Theory for the Sciences</em>; The MIT Press, 2014.</li>
    <li>Brendan Fong, David I. Spivak, <em>An Invitation to Applied Category Theory: Seven Sketches in Compositionality</em>; Cambridge University Press, 2019.</li>
</ul>


Our <code>data_algebra</code> cateogry <code>D</code> is defined as follows.

<ul>
    <li>The objects of our category are single table <a href="https://en.wikipedia.org/wiki/Database_schema">schemas</a>.</li>
    <li>The arrows of our category are <code>data_algebra</code> operator chains.</li>
    <li>Composition of arrows in our category is query commposition.  We will demonstarate query composition in a bit, but as a hint it is not function composition or list concatination.</li>
</ul>

Let's demonstrate the above with <code>Python</code> code.  The <code>data_algebra</code> allows for the specification of data transforms.

In [3]:
from data_algebra.data_ops import *
import pandas

d = pandas.DataFrame({
    'x': [1, 2, 3],
    'y': [3, 4, 4],
})

d

,x,y
0,1,3
1,2,4
2,3,4


To specify adding a new derived column <code>z</code> we would write code such as the following.

In [9]:
td = describe_table(d)

a = td.extend(
    { 'z': 'x.mean()' },
    partition_by=['y']
)

a

TableDescription(
 table_name='data_frame',
 column_names=[
   'x', 'y']) .\
   extend({
    'z': 'x.mean()'},
   partition_by=['y'])

We can let this transform act on data.

In [10]:
a.transform(d)

,x,y,z
0,1,3,1.0
1,2,4,2.5
2,3,4,2.5


Or we can compose this transform with more operations to create a composite transform.

In [11]:
b = a.extend({
    'ratio': 'y / x'
})

b

TableDescription(
 table_name='data_frame',
 column_names=[
   'x', 'y']) .\
   extend({
    'z': 'x.mean()'},
   partition_by=['y']) .\
   extend({
    'ratio': 'y / x'})

As a bonus we can also map the above transform to a <a href="https://en.wikipedia.org/wiki/SQL"><code>SQL</code></a> query representing the same action in databases.

In [17]:
from data_algebra.SQLite import SQLiteModel

print(
    b.to_sql(db_model=SQLiteModel(), pretty=True)
)

SELECT "x",
       "y",
       "z",
       "y" / "x" AS "ratio"
FROM
  (SELECT "x",
          "y",
          avg("x") OVER (PARTITION BY "y") AS "z"
   FROM ("data_frame") "SQ_0") "SQ_1"


All of this is the convenient interface we expect users will want.  However, if we asked that all operators specified their expected input schema (or their domain) we have the cateogry <code>D</code>.  We don't expect users to do such, but we have code supporting this style of notation to show that the <code>data_algebra</code> is in fact related to a nice category over schemas.

Lets re-write the above queries as formal category arrows.

In [22]:
from data_algebra.arrow import *

a1 = DataOpArrow(a)

print(str(a1))

[
 'data_frame':
  [ x: <class 'numpy.int64'>, y: <class 'numpy.int64'> ]
   ->
  [ x, y, z ]
]



The above is rendering the arrow as just its domain and co-domain. The doman and co-domains are just single-table schemas: lists of column names (possibly with column types).

We can get a more detailed representation of the arrow as follows.

In [26]:
print(a1.__repr__())

DataOpArrow(
 TableDescription(
 table_name='data_frame',
 column_names=[
   'x', 'y']) .\
   extend({
    'z': 'x.mean()'},
   partition_by=['y']),
 free_table_key='data_frame')


Or we can examine the domain and co-domain directly.  Here we are using a common category theory trick: associating the object with the identity arrow of the object.  So what we are showing as domain and co-domains are actually identity arrows instead of objets.

In [23]:
a1.dom()

DataOpArrow(
 TableDescription(
 table_name='',
 column_names=[
   'x', 'y']),
 free_table_key='')

In [24]:
a1.cod()

DataOpArrow(
 TableDescription(
 table_name='',
 column_names=[
   'x', 'y', 'z']),
 free_table_key='')

Now we can write our second transform step as an arrow as follows.

In [29]:
a2 = a1.outgoing_table_description().extend({
    'ratio': 'y / x'
})

a2

TableDescription(
 table_name='',
 column_names=[
   'x', 'y', 'z']) .\
   extend({
    'ratio': 'y / x'})

They payoff is, of course, arrow composition.

In [30]:
composite = a1 >> a2

TypeError: transform() got an unexpected keyword argument 'strict'

In [31]:
type(a1)

data_algebra.arrow.DataOpArrow